# Setup

In [ ]:
import pandas as pd
import numpy as np

import json
import matplotlib.pyplot as plt
import datetime
from datasets import Dataset

# Load Data

In [ ]:
df_HoC_1990s_raw = pd.read_csv('/kaggle/input/parlspeech/df_HoC_1990s.csv')
df_HoC_1990s = df_HoC_1990s_raw[['date', 'agenda', 'speechnumber', 'speaker', 'party', 'text', 'terms']]

print(df_HoC_1990s_raw.columns)
del df_HoC_1990s_raw

In [ ]:
df_HoC_2000s_raw = pd.read_csv('/kaggle/input/parlspeech/df_HoC_2000s.csv')
df_HoC_2000s = df_HoC_2000s_raw[['date', 'agenda', 'speechnumber', 'speaker', 'party', 'text', 'terms']]

print(df_HoC_2000s_raw.columns)
del df_HoC_2000s_raw

In [ ]:
df_HoC_2000s.head(3)

# Exploratory Data Analaysis

In [ ]:
# Most frequent speaker
list_topSpeakers = df_HoC_2000s['speaker'].value_counts().head(10).index.tolist()
list_topSpeakers

In [ ]:
# Find Agendas with all the top 3 speakres
df_HoC_2000s[df_HoC_2000s['speaker'].isin(['David Cameron', 'Theresa May', 'Tony Blair'])].groupby('agenda').filter(lambda x: set(['David Cameron', 'Theresa May', 'Tony Blair']).issubset(set(x['speaker'])))['agenda'].unique()

In [ ]:
# David Cameron Last speech
print(df_HoC_2000s[df_HoC_2000s['speaker'] == 'David Cameron'].sort_values(by='date', ascending=False)['date'].head(1))
print('\n')

# Theresa May Last speech
print(df_HoC_2000s[df_HoC_2000s['speaker'] == 'Theresa May'].sort_values(by='date', ascending=False)['date'].head(1))

In [ ]:
print(f"David Cameron has {df_HoC_2000s[df_HoC_2000s['speaker'] == 'David Cameron']['terms'].sum()} terms")
print(f"Boris Johnson has {df_HoC_2000s[df_HoC_2000s['speaker'] == 'Boris Johnson']['terms'].sum()} terms")

In [ ]:
# Summary of the 'agenda'
filtered_df = df_HoC_2000s[~df_HoC_2000s['agenda'].str.contains('\[', case=False, na=False)]

agenda_summary = filtered_df.groupby('agenda').agg(
    num_speakers=('speaker', 'nunique'),
    n_rows = ('agenda', 'size'),
    start_date=('date', 'min'),
    end_date=('date', 'max')
).reset_index()

#agenda_summary

# Filter for agendas with N-speakers & N-rows
agenda_summary[
    (agenda_summary['num_speakers'] <= 10) & 
    (agenda_summary['num_speakers'] >= 5) & 
    (agenda_summary['start_date'] > '2015-01-01') & 
    (agenda_summary['n_rows'] >= 20) &
    (agenda_summary['agenda'].str.contains('EU', case=False, na=False))
].sort_values(by='num_speakers', ascending=False)#.head(50)#.iloc[3]['agenda']

In [ ]:
# Speakers in the 'Free Movement of EU Nationals' agenda
miniDebate_speakers = df_HoC_2000s[df_HoC_2000s['agenda'].str.contains('Free Movement of EU Nationals', case=False, na=False)]['speaker'].unique()#.iloc[0]['text']
print(miniDebate_speakers)

# Print the number of rows that each of the speakers has in df_HoC_2000s
for speaker in miniDebate_speakers:
    print(speaker, df_HoC_2000s[df_HoC_2000s['speaker'] == speaker].shape[0])

In [ ]:
df_HoC_1990s[df_HoC_1990s['speaker'] == 'Theresa May']

# Create Subsets of Data

## Extract a Minidebate Session

In [ ]:
df_HoC_2005 = df_HoC_2000s[df_HoC_2000s['date'].str.contains('2005')]
df_HoC_2015 = df_HoC_2000s[df_HoC_2000s['date'].str.contains('2015')]

In [ ]:
df_HoC_miniDebate = df_HoC_2000s[df_HoC_2000s['agenda'].str.contains('Free Movement of EU Nationals', case=False, na=False)]
df_HoC_miniDebate.to_csv('/kaggle/working/df_HoC_miniDebate.csv', index=False)
df_HoC_miniDebate

## Extract Inferencing Pairs

In [ ]:
prev_speaker = df_HoC_2000s["speaker"].shift(1)
prev_text = df_HoC_2000s["text"].shift(1)
prev_date = df_HoC_2000s["date"].shift(1)
prev_agenda = df_HoC_2000s["agenda"].shift(1)
prev_terms = df_HoC_2000s["terms"].shift(1)
prev_party = df_HoC_2000s["party"].shift(1)

In [ ]:
mask = (
    (df_HoC_2000s["speaker"] == "David Cameron") &
    (prev_speaker != "David Cameron") &
    (df_HoC_2000s["date"] == prev_date) &
    (df_HoC_2000s["agenda"] == prev_agenda)
)
filtered_df = df_HoC_2000s[mask]

# Create prompt-response pairs
prompt_response_pairs = filtered_df[['text', 'terms', 'agenda']].copy()
prompt_response_pairs['prompt'] = prev_text[mask]
prompt_response_pairs['prompt_terms'] = prev_terms[mask]
prompt_response_pairs['prompt_speaker'] = prev_speaker[mask]
prompt_response_pairs['prompt_agenda'] = prev_agenda[mask]
prompt_response_pairs['prompt_date'] = prev_date[mask]
prompt_response_pairs['prompt_party'] = prev_party[mask]
prompt_response_pairs = prompt_response_pairs.rename(columns={'text': 'response', 
                                                              'terms': 'response_terms', 
                                                              'agenda': 'response_agenda', 
                                                              'date': 'response_date'})

# Filter out rows where prompt_terms and response_terms are less than 50
prompt_response_pairs = prompt_response_pairs[
    (prompt_response_pairs['prompt_terms'] >= 50) & 
    (prompt_response_pairs['response_terms'] >= 50)
]

# Modify the prompt to include the speaker name, party name, and "said: " before the text
prompt_response_pairs['prompt'] = prompt_response_pairs.apply(
    lambda row: f"{row['prompt_speaker']} ({row['prompt_party']}) said: {row['prompt']}", axis=1
)

# Select and reorder columns
df_pairs_DavidCameron = prompt_response_pairs.reset_index(drop=True)
df_pairs_DavidCameron = df_pairs_DavidCameron[['prompt_date', 'prompt_agenda', 'prompt_speaker', 'prompt', 'prompt_terms', 'response', 'response_terms']]

df_pairs_DavidCameron.to_csv('/kaggle/working/df_pairs_DavidCameron.csv', index=False)

In [ ]:
prev_speaker = df_HoC_1990s["speaker"].shift(1)
prev_text = df_HoC_1990s["text"].shift(1)
prev_date = df_HoC_1990s["date"].shift(1)
prev_agenda = df_HoC_1990s["agenda"].shift(1)
prev_terms = df_HoC_1990s["terms"].shift(1)
prev_party = df_HoC_1990s["party"].shift(1)

In [ ]:
mask = (
    (df_HoC_1990s["speaker"] == "Theresa May") &
    (prev_speaker != "Theresa May") &
    (df_HoC_1990s["date"] == prev_date) &
    (df_HoC_1990s["agenda"] == prev_agenda)
)
filtered_df = df_HoC_1990s[mask]

# Create prompt-response pairs
prompt_response_pairs = filtered_df[['text', 'terms', 'agenda']].copy()
prompt_response_pairs['prompt'] = prev_text[mask]
prompt_response_pairs['prompt_terms'] = prev_terms[mask]
prompt_response_pairs['prompt_speaker'] = prev_speaker[mask]
prompt_response_pairs['prompt_agenda'] = prev_agenda[mask]
prompt_response_pairs['prompt_date'] = prev_date[mask]
prompt_response_pairs['prompt_party'] = prev_party[mask]
prompt_response_pairs = prompt_response_pairs.rename(columns={'text': 'response', 
                                                              'terms': 'response_terms', 
                                                              'agenda': 'response_agenda', 
                                                              'date': 'response_date'})

# Filter out rows where prompt_terms and response_terms are less than 50
prompt_response_pairs = prompt_response_pairs[
    (prompt_response_pairs['prompt_terms'] >= 10) & 
    (prompt_response_pairs['response_terms'] >= 10)
]

# Modify the prompt to include the speaker name, party name, and "said: " before the text
prompt_response_pairs['prompt'] = prompt_response_pairs.apply(
    lambda row: f"{row['prompt_speaker']} ({row['prompt_party']}) said: {row['prompt']}", axis=1
)

# Select and reorder columns
df_pairs_TheresaMay = prompt_response_pairs.reset_index(drop=True)
df_pairs_TheresaMay = df_pairs_TheresaMay[['prompt_date', 'prompt_agenda', 'prompt_speaker', 'prompt', 'prompt_terms', 'response', 'response_terms']]

df_pairs_TheresaMay.to_csv('/kaggle/working/df_pairs_TheresaMay.csv', index=False)



In [ ]:
df_pairs_TheresaMay